In [45]:
import pandas as pd
import sqlite3
import datetime
import os

In [48]:
os.listdir('/Users/sdale/repos/find_my_tie')

['scraping',
 'catelog_ties',
 '.DS_Store',
 'catalog_ties.ipynb',
 'requirements.txt',
 'dataset_v0',
 'scalable_instructions.txt',
 'create_tree.ipynb',
 'temp.ipynb',
 '.gitignore',
 'output.png',
 'dataset_v0.1',
 'color_grabber',
 'venv',
 'color_grabber_nn',
 '.git',
 'semantic_segmentation']

In [33]:
DB_PATH = '/Users/sdale/repos/findmytie_website/findmytie_backend/db.sqlite3' 
def get_table_names():
    conn = sqlite3.connect(DB_PATH)
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    conn.close()
    return tables

TABLE_NAMES = {
    'listing': 'api_listing', 
    'search_query': 'api_searchquery',
    'query_match': 'api_querymatch',
}
def get_data():
    conn = sqlite3.connect(DB_PATH)
    df_dict = {}
    for table, table_key in TABLE_NAMES.items():
        query = f"SELECT * FROM {table_key}"
        df = pd.read_sql_query(query, conn)
        df_dict[table] = df
    conn.close()
    return df_dict

def put_data(df_dict: dict):
    conn = sqlite3.connect(DB_PATH)
    for table, df in df_dict.items():
        table_key = TABLE_NAMES[table]
        df.to_sql(table_key, conn, if_exists='replace', index=False)
    conn.close()

In [34]:
df_dict = get_data()

In [35]:
# remove all matches
put_data({'query_match': pd.DataFrame(columns=df_dict['query_match'].columns)})

In [36]:
df_dict['listing'].head()

,title,solid_color,color_description,colors,price,retailer_url,retailer_image_url,image,created_at,id
0,RBOCOTT Solid Color Tie Formal Necktie for Men,1,Baby Blue,None,8.49,https://amzn.to/3OvN9ED,https://m.media-amazon.com/images/I/81Xijyo9gs...,images/81Xijyo9gsL._AC_SY879_.jpg,2024-11-30 15:00:42.352728,1
1,RBOCOTT Solid Color Tie Formal Necktie for Men,1,Black,None,7.19,https://amzn.to/3OF2dzv,https://m.media-amazon.com/images/I/81onvueUmZ...,images/81onvueUmZL._AC_SY879_.jpg,2024-11-30 15:00:42.352728,2
2,RBOCOTT Solid Color Tie Formal Necktie for Men,1,Blush,None,8.49,https://amzn.to/4idgtgA,https://m.media-amazon.com/images/I/812pABcFwS...,images/812pABcFwSL._AC_SY879_.jpg,2024-11-30 15:00:42.352728,3
3,RBOCOTT Solid Color Tie Formal Necktie for Men,1,Bright Sage,None,7.99,https://amzn.to/3VfTrMn,https://m.media-amazon.com/images/I/81IheLda7o...,images/81IheLda7oL._AC_SY879_.jpg,2024-11-30 15:00:42.352728,4
4,RBOCOTT Solid Color Tie Formal Necktie for Men,1,Burgundy,None,7.99,https://amzn.to/3VgqWOA,https://m.media-amazon.com/images/I/81JrFY9rwz...,images/81JrFY9rwzL._AC_SY879_.jpg,2024-11-30 15:00:42.352728,5


In [37]:
ml_df = pd.read_csv('manual_listings.csv')

In [38]:
ml_df.head()

,title,solid_color,color_description,colors,price,retailer_url,retailer_image_url,internal_image_path
0,RBOCOTT Solid Color Tie Formal Necktie for Men,True,Baby Blue,NaN,8.49,https://amzn.to/3OvN9ED,https://m.media-amazon.com/images/I/81Xijyo9gs...,media/images/81Xijyo9gsL._AC_SY879_.jpg
1,RBOCOTT Solid Color Tie Formal Necktie for Men,True,Black,NaN,7.19,https://amzn.to/3OF2dzv,https://m.media-amazon.com/images/I/81onvueUmZ...,media/images/81onvueUmZL._AC_SY879_.jpg
2,RBOCOTT Solid Color Tie Formal Necktie for Men,True,Blush,NaN,8.49,https://amzn.to/4idgtgA,https://m.media-amazon.com/images/I/812pABcFwS...,media/images/812pABcFwSL._AC_SY879_.jpg
3,RBOCOTT Solid Color Tie Formal Necktie for Men,True,Bright Sage,NaN,7.99,https://amzn.to/3VfTrMn,https://m.media-amazon.com/images/I/81IheLda7o...,media/images/81IheLda7oL._AC_SY879_.jpg
4,RBOCOTT Solid Color Tie Formal Necktie for Men,True,Burgundy,NaN,7.99,https://amzn.to/3VgqWOA,https://m.media-amazon.com/images/I/81JrFY9rwz...,media/images/81JrFY9rwzL._AC_SY879_.jpg


In [39]:
print(sorted(ml_df.columns))
print(sorted(df_dict['listing'].columns))

['color_description', 'colors', 'internal_image_path', 'price', 'retailer_image_url', 'retailer_url', 'solid_color', 'title']
['color_description', 'colors', 'created_at', 'id', 'image', 'price', 'retailer_image_url', 'retailer_url', 'solid_color', 'title']


In [40]:
ml_df['image'] = ml_df['internal_image_path'].apply(lambda x: x.replace('media/', ''))
ml_df.drop(columns=['internal_image_path'], inplace=True)
ml_df['created_at'] = datetime.datetime.now()
ml_df['id'] = ml_df.index + 1
ml_df['colors'] = ''

In [44]:
datetime.datetime.now()

datetime.datetime(2024, 11, 30, 16, 18, 53, 688575)

In [42]:
print(sorted(ml_df.columns))
print(sorted(df_dict['listing'].columns))

['color_description', 'colors', 'created_at', 'id', 'image', 'price', 'retailer_image_url', 'retailer_url', 'solid_color', 'title']
['color_description', 'colors', 'created_at', 'id', 'image', 'price', 'retailer_image_url', 'retailer_url', 'solid_color', 'title']


In [43]:
put_data({'listing': ml_df})

In [ ]:
dt = df_dict['query_match'].iloc[0]['created_at']
print(dt)

In [ ]:
# parse datetime for date and time
dt_obj = datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S.%f')
print(dt_obj)
print(type(dt_obj))
print(type(dt))
print(dt_obj.date())

In [ ]:
# get yesterday's date
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
print(f"Yesterday: {yesterday}")

In [ ]:
# remove rows that are older than yesterday
for table in ('query_match', 'search_query'):
    df = df_dict[table]
    print(f"{table} before: {len(df)}")
    for i, row in df.iterrows():
        dt = row['created_at']
        dt_obj = datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S.%f')
        print(f"\t{dt_obj.date()} - {dt_obj.strftime("%H:%M:%S")} - {dt_obj.date() < yesterday.date()}")
        if dt_obj.date() < yesterday.date():
            df.drop(i, inplace=True)
    print(f"{table} after: {len(df)}")

In [ ]:
# check for dangling references, remove them
dangling_query_matches = []
search_query_ids = df_dict['search_query']['id'].values
for i, row in df_dict['query_match'].iterrows():
    query_id = row['search_query_id']
    if query_id not in search_query_ids:
        dangling_query_matches.append(i)
print(f"found {len(dangling_query_matches)} dangling query matches")
df_dict['query_match'].drop(dangling_query_matches, inplace=True)

In [ ]:
# Close the connection
conn.close()